# Gold layer


Gold = DW (modelo dimensional): publica dimensões e fatos com nomes de negócio (`gold.dim_*`, `gold.fact_*`) e chaves/prefixos corretos (`sk_`, `nk_`, `fk_`).


In [ ]:
USE CATALOG manufatura_lakehouse;
CREATE SCHEMA IF NOT EXISTS gold;


## 1) gold.dim_tempo


### 1.2 Dimensão Equipamento (Snapshot Atual)


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {{fqtn(SCHEMA_GOLD, 'dim_equipamento')}} AS
SELECT 
    equipment_id,
    equipment_name,
    equipment_type,
    location,
    manufacturer,
    model,
    status as current_status,
    installation_date
FROM {{fqtn(SCHEMA_SILVER, 'equipment_clean')}}
""")


In [ ]:
-- ============================================
-- GOLD: dim_tempo (INSERT-ONLY, idempotente)
-- ============================================
-- Range de datas derivado da Silver
CREATE OR REPLACE TEMP VIEW tempo_range AS
SELECT
  COALESCE(MIN(CAST(planned_start AS DATE)), DATE('2019-01-01')) AS dt_min,
  COALESCE(MAX(CAST(planned_start AS DATE)), DATE('2030-12-31')) AS dt_max
FROM silver.production_orders_clean;

-- Série diária
CREATE OR REPLACE TEMP VIEW tempo_series AS
WITH r AS (SELECT dt_min, dt_max FROM tempo_range),
seq AS (SELECT sequence(dt_min, dt_max, INTERVAL 1 DAY) AS dts FROM r)
SELECT explode(dts) AS data FROM seq;

-- Mapeamentos estáveis (sem depender de locale)
CREATE OR REPLACE TEMP VIEW dim_tempo_stage AS
WITH base AS (
  SELECT
    CAST(date_format(data, 'yyyyMMdd') AS INT) AS tempo_sk,
    data,
    YEAR(data)  AS ano,
    QUARTER(data) AS trimestre,
    MONTH(data) AS mes,
    DAY(data)   AS dia,
    CASE WHEN dayofweek(data)=1 THEN 7 ELSE dayofweek(data)-1 END AS dia_semana  -- 1=Seg ... 7=Dom
  FROM tempo_series
)
SELECT
  tempo_sk,
  CAST(date_format(data, 'yyyyMMdd') AS INT) AS date_id,
  data,
  ano,
  trimestre,
  mes,
  dia,
  dia_semana,
  -- nomes PT-BR fixos por mapeamento
  element_at(map(
    1,'janeiro', 2,'fevereiro', 3,'março', 4,'abril', 5,'maio', 6,'junho',
    7,'julho', 8,'agosto', 9,'setembro', 10,'outubro', 11,'novembro', 12,'dezembro'
  ), mes) AS nome_mes,
  element_at(map(
    1,'segunda', 2,'terça', 3,'quarta', 4,'quinta', 5,'sexta', 6,'sábado', 7,'domingo'
  ), dia_semana) AS nome_dia
FROM base;

-- Tabela alvo
CREATE TABLE IF NOT EXISTS gold.dim_tempo (
  tempo_sk   INT,
  date_id    INT,          -- yyyymmdd
  data       DATE,
  ano        INT,
  trimestre  INT,
  mes        INT,
  dia        INT,
  dia_semana INT,          -- 1=Seg ... 7=Dom
  nome_mes   STRING,
  nome_dia   STRING
) USING DELTA;

-- MERGE INSERT-ONLY (não faz UPDATE nunca)
MERGE INTO gold.dim_tempo t
USING dim_tempo_stage s
ON t.tempo_sk = s.tempo_sk
WHEN NOT MATCHED THEN INSERT (
  tempo_sk, date_id, data, ano, trimestre, mes, dia, dia_semana, nome_mes, nome_dia
) VALUES (
  s.tempo_sk, s.date_id, s.data, s.ano, s.trimestre, s.mes, s.dia, s.dia_semana, s.nome_mes, s.nome_dia
);


### 1.2 Dimensão Equipamento (Snapshot Atual)


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {fqtn(SCHEMA_GOLD, 'dim_equipamento')} AS
SELECT 
    equipment_id,
    equipment_name,
    equipment_type,
    location,
    manufacturer,
    model,
    status as current_status,
    installation_date
FROM {fqtn(SCHEMA_SILVER, 'equipment_clean')}
""")


## 2) gold.dim_equipamento e dim_equipamento_scd

As dimensões de equipamento serão criadas a partir da SCD da Silver. Ver arquivos:
- `models/gold/dim_equipamento.dbquery.ipynb`
- `models/gold/dim_equipamento_scd.dbquery.ipynb`


### 1.3 Dimensão Produto


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {{fqtn(SCHEMA_GOLD, 'dim_produto')}} AS
SELECT DISTINCT
    product_id as product_name
FROM {{fqtn(SCHEMA_SILVER, 'production_orders_clean')}}
WHERE product_id IS NOT NULL
""")


### 1.4 Dimensão Técnico


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {{fqtn(SCHEMA_GOLD, 'dim_tecnico')}} AS
SELECT DISTINCT
    technician_id as technician_name
FROM {{fqtn(SCHEMA_SILVER, 'maintenance_orders_clean')}}
WHERE technician_id IS NOT NULL
""")


### 1.5 Dimensão Tipo Manutenção


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {{fqtn(SCHEMA_GOLD, 'dim_tipo_manutencao')}} AS
SELECT DISTINCT
    maintenance_type as description
FROM {{fqtn(SCHEMA_SILVER, 'maintenance_orders_clean')}}
WHERE maintenance_type IS NOT NULL
""")


### 1.6 Dimensão Defeito


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {{fqtn(SCHEMA_GOLD, 'dim_defeito')}} AS
SELECT DISTINCT
    defect_codes as defect_code, defect_codes as defect_description
FROM {{fqtn(SCHEMA_SILVER, 'quality_inspections_clean')}}
WHERE defect_codes IS NOT NULL AND defect_codes <> ''
""")


## 3) gold.dim_produto, dim_tecnico, dim_tipo_manutencao, dim_defeito

As demais dimensões serão criadas nos arquivos .dbquery.ipynb correspondentes.


## 4) gold.fact_producao, fact_manutencao, fact_qualidade, fact_iot_agregado

Os fatos serão criados nos arquivos .dbquery.ipynb correspondentes.


## 5) Views Analíticas OEE

As views analíticas serão criadas nos arquivos .dbquery.ipynb correspondentes:
- `vw_oee_diario` - OEE diário por equipamento
- `vw_downtime_por_causa` - Downtime agregado por causa
- `vw_equipamentos_criticos` - Equipamentos com maior downtime ou menor OEE
- `vw_tendencias_sensores` - Tendências de leituras de sensores
